# Phacomochere obfuscation

In this notebook, we'll look at how we can generate obfuscations from two different obfuscators on a simple C++ function.

* [Pluto](https://github.com/bluesadi/Pluto)
  * LLVM based obfuscator
  * Obfuscate IR (.ll)
  * [llvm-ob-passes](https://github.com/expend20/llvm-ob-passes) add the work on passes manager like eshard-ollvm

* [Tigress](https://tigress.wtf/)
  * Source to source based obfuscator
  * Work on multiple arch (intel/arm/wasm, 32/64 bits ...)
  * Easy to use and well documented

## Simple C function

In [ ]:
#include <stdio.h>
#include <stdint.h>
#include <stdbool.h>
#include <stdlib.h>
#include <string.h>

char* good_string = (char*)"Phacomochere\0";
int size_good_string = 0;

uint64_t good_value = 0x6964654A654C6F59;

static bool RootFunction(char* string, uint64_t value)
{
    if (string == NULL)
        return false;

    size_good_string = strlen(good_string) + 1;
 
    for (int i = 0; i < size_good_string; ++i)
    {
        if (string[i] != good_string[i])
            return false;
    }

    if (value != good_value)
        return false;

    return true;
}

int main(int argc, char *argv[])
{
    size_good_string = strlen(good_string) + 1;

    if (argc != 3) {
        fprintf(stderr, "Usage: %s <string> <hex_value>\n", argv[0]);
        return 1;
    }

    uint64_t value = strtoull(argv[2], NULL, 16);

    if (RootFunction(argv[1], value)) {
        printf("RootFunction returned true.\n");
    } else {
        printf("RootFunction returned false.\n");
    }

    return 0;
}

## LLVM Based Obfuscator(s)

First, we'll look at pluto obfuscator, and more explictly a wrapper of it named **llvm-ob-passes**:
* Installation
* Usage

Note that i already compiled libPasses, find them in [bin](./bin/libpasses.zip) directory

### llvm-ob-passes installation

In [ ]:
#!/bin/bash

set -e

# Set variables
LLVM_VERSIONS=("16" "17" "18") # or you can also use $(llvm-config --version | cut -d'.' -f1) to get installed version
ARTIFACTS_DIR="artifacts"
RELEASE_DIR="release_files"

# Install necessary packages
sudo apt-get update
sudo apt-get install -y git wget libseccomp-dev zip

# Clone repository and submodules
git clone --recurse-submodules https://github.com/expend20/llvm-ob-passes
cd llvm-ob-passes

# Main build loop
for LLVM_VERSION in "${LLVM_VERSIONS[@]}"; do
  # Install LLVM
  wget https://apt.llvm.org/llvm.sh && chmod +x llvm.sh
  sudo ./llvm.sh "$LLVM_VERSION"

  # Update clang and clang++ links
  sudo ln -sf /usr/bin/clang-"$LLVM_VERSION" /usr/bin/clang
  sudo ln -sf /usr/bin/clang++-"$LLVM_VERSION" /usr/bin/clang++
  sudo ln -sf /usr/bin/opt-"$LLVM_VERSION" /usr/bin/opt

  # Display version information
  clang --version
  opt --version

  # Configure and build with CMake
  cmake -B build -DLLVM_DIR=/usr/lib/llvm-"$LLVM_VERSION"/cmake \
    -DCMAKE_C_COMPILER=clang-"$LLVM_VERSION" \
    -DCMAKE_CXX_COMPILER=clang++-"$LLVM_VERSION"
  cmake --build build

  # Collect artifacts
  mkdir -p "$ARTIFACTS_DIR"
  cp build/libpasses.so "$ARTIFACTS_DIR/libpasses-$LLVM_VERSION.so"
  cp build/wrapper "$ARTIFACTS_DIR/wrapper-$LLVM_VERSION"
done

# Prepare release files
mkdir -p "$RELEASE_DIR"
cp "$ARTIFACTS_DIR"/libpasses-*.so "$RELEASE_DIR/"
cp "$ARTIFACTS_DIR/wrapper-18" "$RELEASE_DIR/wrapper"

# Package into a zip file
cd "$RELEASE_DIR"
zip -r ../libpasses.zip *
cd ..

### Usage

/!\ All string between <> have to be replaced by something of your choice. /!\ <br>

**An automatic compilation script is present in bash directory**

In [ ]:
#!/bin/bash

clang -c -S -emit-llvm -O1 main.c -o dump/pluto/main.ll

# I recommend using llvm 16, because 17 and 18 used to segfault a lot
opt -load-pass-plugin=./lib/libpasses-$(llvm-config --version | cut -d'.' -f1).so -passes "example-pass" dump/pluto/main.ll -S -o "dump/pluto/example/example.ll" -debug-pass-manager

clang dump/pluto/main_opt.ll -o dump/pluto/main_opt

#### Passes name
* example-pass
* pluto-bogus-control-flow
* pluto-flattening
* pluto-mba-obfuscation
* pluto-substitution
* pluto-global-encryption
* pluto-indirect-call

## Tigress

### Opaque Predicate

- [executable](opaquePredicate.d/initOpaque)
- [obfuscated_source](opaquePredicate.d/initOpaque.c)

#### Graph view
![](../resources/images/pseudocode/initopaque.png)

##### Command
```shell
tigress \
   --Environment=x86_64:Linux:Gcc:11.4.0 \
   --Transform=InitOpaque \
   --Functions=* \
   obfuscation-demo.c \
   --out=tigress/code/opaquePredicate.d/initOpaque.c
```

## Mixed boolean arithmetic

- [executable](mba.d/mba)
- [obfuscated_source](mba.d/mba.c)

#### Graph view
![](../resources/images/pseudocode/mba.png)

##### Command
```shell
tigress \
  --Environment=x86_64:Linux:Gcc:11.4.0 \
  --Transform=EncodeArithmetic \
  --Functions=* \
  --EncodeArithmeticKinds=* \
  obfuscation-demo.c \
  --out=tigress/code/mba.d/mba.c
```

### Flattening

- [executable](flattening.d/flattening)
- [obfuscated_source](flattening.d/flattening.c)

#### Graph view
![](../resources/images/pseudocode/flattening.png)


##### Command
```shell
tigress \
  --Environment=x86_64:Linux:Gcc:11.4.0 \
  --Transform=Flatten \
  --Functions=* \
  --FlattenSplitBasicBlocks=true \
  --FlattenRandomizeBlocks=true \
  obfuscation-demo.c \
  --out=tigress/code/flattening.d/flattening.c
```

### Bogus Control Flow

- [executable](bogus.d/antiAliasAnalysis)
- [obfuscated_source](bogus.d/antiAliasAnalysis.c)

#### Graph view
![](../resources/images/pseudocode/bogus.png)

##### Command
```shell
tigress \
  --Environment=x86_64:Linux:Gcc:11.4.0 \
  --Transform=Flatten \
  --Functions=* \
  --AntiAliasAnalysisObfuscateIndex=true \
  --AntiAliasAnalysisBogusEntries=true \
    obfuscation-demo.c \
  --out=tigress/code/bogus.d/antiAliasAnalysis.c
```

### Results

All results are stored into the `dump` directory.